In [2]:
# !wget https://huggingface.co/datasets/mesolitica/llama3-70b-qa/resolve/main/generated-wikipedia-qa-llama3.jsonl

In [3]:
import json
import random
from tqdm import tqdm
from malaysian_sft import accept

In [4]:
wikipedia = []
with open('generated-wikipedia-qa-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        lower = l['text'].lower()
        
        for q in l['questions']:
            q_low = q['question'].lower()
            if ' ini' in q_low:
                continue
            
            if 'dalam teks' in q_low:
                continue
                
            if 'berdasarkan teks' in q_low:
                continue
                
            if len(q_low.split()) <= 4:
                continue
        
            wikipedia.append(q)
            
len(wikipedia)

480746

In [5]:
wikipedia[10000]

{'question': 'Nama lapangan terbang yang mula beroperasi secara aktif pada akhir tahun 2005?',
 'A': 'Lapangan Terbang Tabing',
 'B': 'Lapangan Terbang Minangkabau',
 'C': 'Lapangan Terbang Padang',
 'D': 'Lapangan Terbang Bukittinggi',
 'answer': 'B'}

In [6]:
prompts = [
    [
        'Please provide the correct answer only',
        'Sila berikan jawapan sahaja',
    ],
    [
        'Sila berikan jawapan penuh',
        'please provide the full answer',
    ]
]

wikipedia_instructions = []
for w in tqdm(wikipedia):
    
    success = True
    for k in ['question', 'A', 'B', 'C', 'D']:
        if len(w[k]) < 1:
            success = False
            break
    
    if not success:
        continue
            
    i = f"""
{w['question']}
A. {w['A']}
B. {w['B']}
C. {w['C']}
D. {w['D']}
""".strip()
    
    choice = random.randint(0, 1)
    p = random.choice(prompts[choice])
    
    if choice:
        try:
            d = {
                'prompt_input': None,
                'input': f'{p}\n\n{i}',
                'output': f"{w['answer']}. {w[w['answer']]}",
            }
            wikipedia_instructions.append(d)
        except:
            pass
        
    else:
        try:
            d = {
                'prompt_input': None,
                'input': f'{p}\n\n{i}',
                'output': f"{w['answer']}",
            }
            wikipedia_instructions.append(d)
        except:
            pass

100%|███████████████████████████████████████████████████████████████████████████| 480746/480746 [00:01<00:00, 314124.48it/s]


In [9]:
wikipedia_instructions[2]

{'prompt_input': None,
 'input': 'Please provide the correct answer only\n\nBahasa Melayu Klasik ialah bahasa piawai sastera bersusurgalur daripada kesultanan?\nA. Melaka dan Johor\nB. Riau dan Sumatra\nC. Pattani dan Kelantan\nD. Brunei dan Sabah',
 'output': 'A'}

In [11]:
import pandas as pd

df = pd.DataFrame(wikipedia_instructions)
df.to_parquet('prepared-llama3-70b-qa.parquet')

In [12]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-llama3-70b-qa.parquet",
    path_in_repo='data/llama3_70b_qa-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-llama3-70b-qa.parquet:   0%|          | 0.00/39.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/1e164b5ccb7413ca4124052237234805d6448c5a', commit_message='Upload data/llama3_70b_qa-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='1e164b5ccb7413ca4124052237234805d6448c5a', pr_url=None, pr_revision=None, pr_num=None)